# 1. 필요한 정보가 들어있는 웹사이트 찾기

<img src="image_lecture/crawling_01.png">

<img src="image_lecture/crawling_02.png">

# 2. 웹사이트 구조 파악

In [ ]:
url = 'https://finance.naver.com/item/sise_day.nhn?code=005930'

<img src="image_lecture/crawling_03.png">

# 3. 필요한 정보의 위치 파악

<img src="image_lecture/crawling_04.png">

# 4. BeautifulSoup 등으로 필요한 정보 추출

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
source = urlopen(url).read()
soup = BeautifulSoup(source, 'lxml')
soup

In [ ]:
soup.find_all('span', class_='tah p10 gray03')   # 날짜 수집   

In [ ]:
# /html/body/table[1]/tbody/tr[3]/td[2]/span
soup.find_all('td', class_='num')   # 종가 수집

In [ ]:
# 너무 많으니 몇 개나 있나 세어보자
len(soup.find_all('td', class_='num'))

In [ ]:
# 6의 배수마다 종가가 산출되는 것 같으니 확인
for i in range(0, 60, 6):
    print(soup.find_all('td', class_='num')[i])

# 5. 추출한 정보 정제 

In [ ]:
for i in range(10):
    d = soup.find_all('span', class_='tah p10 gray03')[i].text
    print(d)

In [ ]:
import pandas as pd

In [ ]:
for i in range(10):
    d = soup.find_all('span', class_='tah p10 gray03')[i].text
    d = pd.to_datetime(d).date()
    print(d)

In [ ]:
for i in range(0, 60, 6):
    p = soup.find_all('td', class_='num')[i].text
    print(p)

In [ ]:
type(p)

In [ ]:
for i in range(0, 60, 6):
    p = soup.find_all('td', class_='num')[i].text
    p = p.replace(',', '')
    p = float(p)
    print(p)

In [ ]:
type(p)

# 6. 정제된 정보를 데이터화<br>
(데이터프레임 등으로 저장)

In [ ]:
date = []
for i in range(10):
    d = soup.find_all('span', class_='tah p10 gray03')[i].text
    d = pd.to_datetime(d).date()
    date.append(d)
date

In [ ]:
price = []
for i in range(0, 60, 6):
    p = soup.find_all('td', class_='num')[i].text
    p = p.replace(',', '')
    p = float(p)
    price.append(p)
price

In [ ]:
price[5]

In [ ]:
prices = {}
for i in range(10):
    prices[date[i]] = price[i]

In [ ]:
prices

In [ ]:
# Series로 만들기
pd.Series(prices)

In [ ]:
# DataFrame으로 만들기
pd.DataFrame(price, index=date)

함수로 만들기

In [ ]:
# 날짜 포맷 변환
def date_format(d=''):
    if d != '':
        this_date = pd.to_datetime(d).date()
    else:
        this_date = pd.Timestamp.today().date()   # 입력값이 없으면 오늘 날짜를 지정
    return (this_date)

In [ ]:
# 일자별 시세 수집
def stock_prices(stock_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    # 날짜 세팅
    end_date = date_format(end_date)
    if start_date == '':
        start_date = end_date - pd.DateOffset(months=1)
    start_date = date_format(start_date)
    
    # 크롤링
    naver_stock = 'http://finance.naver.com/item/sise_day.nhn?code=' + stock_cd + '&page=' + str(page_n)
    source = urlopen(naver_stock).read()
    soup = BeautifulSoup(source, 'lxml')
    
    dates = soup.find_all('span', class_='tah p10 gray03')   # 날짜 수집   
    prices = soup.find_all('td', class_='num')   # 종가 수집
    
    # 데이터 정제
    for n in range(len(dates)):
    
        if len(dates) > 0:
            
            # 날짜 처리
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                this_close = prices[n*6].text
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
                              
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료
                return historical_prices              
            
    # 페이지 네비게이션
    # last_page 정보 찾기
    if last_page == 0:
        last_page = soup.find_all('table')[1].find('td', class_='pgRR').find('a')['href']
        last_page = last_page.split('&')[1]
        last_page = last_page.split('=')[1]
        last_page = float(last_page)
        
    # 마지막 페이지가 아니면 다음 페이지 호출
    if page_n < last_page:
        page_n = page_n + 1
        stock_prices(stock_cd, start_date, end_date, page_n, last_page)   
        
    return historical_prices  

In [ ]:
# 여러 종목 수집
'''
005930	삼성전자
000660	SK하이닉스
066570	LG전자
'''
stocks = ['005930', '000660', '066570']

In [ ]:
k10_historical_prices = dict()

for stock_cd in stocks:
    
    historical_prices = dict()
    start_date = '2018-10-1'
    end_date = '2018-12-31'
    stock_prices(stock_cd, start_date, end_date)
    
    k10_historical_prices[stock_cd] = historical_prices

In [ ]:
k10_historical_price = pd.DataFrame(k10_historical_prices)
k10_historical_price.head(3)